<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/llm/OpenAssistantLlama213B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPTQ

In [ ]:
!pip install -q auto-gptq

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "TheBloke/OpenAssistant-Llama2-13B-Orca-8K-3319-GPTQ"
model_basename = "gptq_model-4bit-128g"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

In [ ]:
system_message = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
prompt = "日本語でAIについての文章を書いて"
prompt_template=f'''<|system|>{system_message}</s><|prompter|>{prompt}</s><|assistant|>
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt_template):])

LlamaCpp

In [ ]:
!pip install llama-cpp-python

In [ ]:
# !wget https://huggingface.co/TheBloke/OpenAssistant-Llama2-13B-Orca-8K-3319-GGML/resolve/main/openassistant-llama2-13b-orca-8k-3319.ggmlv3.q2_K.bin -P /content
!wget https://huggingface.co/TheBloke/OpenAssistant-Llama2-13B-Orca-8K-3319-GGML/resolve/main/openassistant-llama2-13b-orca-8k-3319.ggmlv3.q4_1.bin -P /content

In [ ]:
from llama_cpp import Llama

llm = Llama(model_path="/content/openassistant-llama2-13b-orca-8k-3319.ggmlv3.q4_1.bin")

In [ ]:
# prompt
system_message = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
user_prompt = "Please write the presentation about the latest AI within 200 words"
prompt = f"""<|system|>{system_message}</s><|prompter|>{user_prompt}</s><|assistant|>"""

# inference
output = llm(
    prompt,
    temperature=0.1,
    stop=["<|system|>", "<|prompter|>", "<|assistant|>", "</s>"],
    # echo=True,
)
print(output["choices"][0]["text"])

In [ ]:
# プロンプトの準備
prompt = """### Instruction: Please write the presentation about the latest AI within 200 words.
### Response:"""

# 推論の実行
output = llm(
    prompt,
    temperature=0.1,
    stop=["Instruction:", "Input:", "Response:", "\n"],
    echo=True,
)
print(output["choices"][0]["text"])